In [25]:
import nltk
sentences = [
    "Thank your message",
    "to show our words to the doctor,",
    "as his next contract checking, to all of us",
    "Anyway, I believe the team, although bit delay and less communication at recent days, they really tried best for paper and cooperation",
]
# simple context free grammar
grammar = nltk.CFG.fromstring("""
    S -> NP VP | VP | Adv S | S Conj S 
    NP -> Det N | Det N PP | Det Adj N | Det Adj N PP | NP PP | Pronoun | NP Conj NP | Adj NP | N | Det PP | N NP | Det Adj | N Adv
    PP -> P NP
    VP -> V NP | VP PP | V | Adv VP | VP Adv | V Pronoun | V VP | V S
    Det -> 'the' | 'a' | 'your' | 'our' | 'his' | 'all' | 'which' | 'some' | 'their'
    Adj -> 'next' | 'recent' | 'bit' | 'less' 
    Adv -> 'anyway' | 'really'  | 'even' | 'lately'  
    Pronoun -> 'us' | 'they' | 'i' | 'you' | 'he' | 'she' | 'who' | 'it' | 'them' | 'there'
    N -> 'doctor' | 'team' | 'message' | 'words' | 'contract' | 'days' | 'paper' | 'cooperation' | 'delay' | 'communication' | 'thoughts' | 'checking' | 'best' | 'delays'
    P -> 'to' | 'as' | 'for' | 'of' | 'at' | 'with' | 'in' | 'on' | 'by'
    V -> 'show' | 'believe' | 'tried' | 'thank' | 'helped' | 'assist' | 'convey' | 'will' | 'were' | 'did'
    Conj -> 'and' | 'though' | 'although'
""")
stop_words = [ '.', ',']
# create a parser
parser = nltk.ChartParser(grammar)
# parse the sentences
# fix the sentences
# to make them more grammatically correct
# and to match the grammar
def sentence_fixer(sentence):
    replace = {
        "your message": "you for your message,",
        "to show our words": "it helped convey our thoughts",
        "as his": "who will assist all of us with checking the",
        "checking, to all of us": ".",
        "anyway, ": "",
        ", although": " really did their best with the paper,",
        "bit delay": "though there were some delays",
        "at recent days,": "lately.",
        "they really tried best for paper and cooperation": ""
    }
    for key, value in replace.items():
        sentence = sentence.lower().replace(key, value)
    return sentence

sentences_fixed = [sentence_fixer(sentence) for sentence in sentences]

for sentence in sentences_fixed:
    print(f"Parsing: {sentence}")
    tokens = nltk.word_tokenize(sentence)
    tokens = [token.lower() for token in tokens if token not in stop_words]
    print(f"Tokens: {tokens}")
    
    try:
        tree = next(parser.parse(tokens))
        print(tree)
        tree.pretty_print()
    except StopIteration:
        print("No parse tree found.")
    print("\n")


Parsing: thank you for your message,
Tokens: ['thank', 'you', 'for', 'your', 'message']
(S
  (VP
    (V thank)
    (NP (NP (Pronoun you)) (PP (P for) (NP (Det your) (N message))))))
               S                      
               |                       
               VP                     
   ____________|___                    
  |                NP                 
  |       _________|___                
  |      |             PP             
  |      |      _______|____           
  |      NP    |            NP        
  |      |     |        ____|_____     
  V   Pronoun  P      Det         N   
  |      |     |       |          |    
thank   you   for     your     message



Parsing: it helped convey our thoughts to the doctor,
Tokens: ['it', 'helped', 'convey', 'our', 'thoughts', 'to', 'the', 'doctor']
(S
  (NP (Pronoun it))
  (VP
    (VP (V helped) (VP (V convey) (NP (Det our) (N thoughts))))
    (PP (P to) (NP (Det the) (N doctor)))))
                           S      

In [18]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# ChatGPT Paraphraser
device = "cpu"

tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")

model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base").to(device)

def paraphrase(
    question,
    num_beams=5,
    num_beam_groups=5,
    num_return_sequences=3,
    repetition_penalty=10.0,
    diversity_penalty=3.0,
    no_repeat_ngram_size=2,
    max_length=128
):
    input_ids = tokenizer(
        f'Paraphrase: {question}',
        return_tensors="pt", padding="longest",
        max_length=max_length,
        truncation=True,
    ).input_ids.to(device)
    
    outputs = model.generate(
        input_ids, repetition_penalty=repetition_penalty,
        num_return_sequences=num_return_sequences, no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams, num_beam_groups=num_beam_groups,
        max_length=max_length, diversity_penalty=diversity_penalty
    )

    res = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return res

text1 = ["Today is our dragon boat festival, in our Chinese culture, to celebrate it with all safe and great in our lives.",
"Hope you too, to enjoy it as my deepest wishes.",
"Thank your message to show our words to the doctor, as his next contract checking, to all of us.",
"I got this message to see the approved message.",
"In fact, I have received the message from the professor, to show me, this, a couple of days ago.",
"I am very appreciated the full support of the professor, for our Springer proceedings publication",
"“During our final discuss, I told him about the new submission — the one we were waiting since last autumn, but the updates was confusing as it not included the full feedback from reviewer or maybe editor?",
"Anyway, I believe the team, although bit delay and less communication at recent days, they really tried best for paper and cooperation.",
"We should be grateful, I mean all of us, for the acceptance and efforts until the Springer link came finally last week, I think.",
"Also, kindly remind me please, if the doctor still plan for the acknowledgments section edit before he sending again.",
"Because I didn't see that part final yet, or maybe I missed, I apologize if so.",
"Overall, let us make sure all are safe and celebrate the outcome with strong coffee and future targets”"
]

for text in text1:
    print(f"Original: {text}")
    paraphrased = paraphrase(text)
    for p in paraphrased:
        print(f"Paraphrased: {p}")
    print("\n")


Original: Today is our dragon boat festival, in our Chinese culture, to celebrate it with all safe and great in our lives.
Paraphrased: Our Chinese culture celebrates today's dragon boat festival to ensure the safety and prosperity of all who come from this world.
Paraphrased: Chinese culture celebrates today's dragon boat festival to ensure the safety and prosperity of all who come from this world.
Paraphrased: In our Chinese culture, today is the day we celebrate dragons in boats to ensure their safety and happiness.


Original: Hope you too, to enjoy it as my deepest wishes.
Paraphrased: May you also have a good time as my thoughts are with you.
Paraphrased: Let's all have a wonderful time together, dear friends.
Paraphrased: I hope you are too, and enjoy it as well.


Original: Thank your message to show our words to the doctor, as his next contract checking, to all of us.
Paraphrased: As the doctor conducts his next contract check, I appreciate your message of thanks for sharing m

chatgpt_paraphraser:
  author={Vladimir Vorobev, Maxim Kuznetsov},
  title={A paraphrasing model based on ChatGPT paraphrases},
  year={2023}


In [19]:
# Grammarly Coedit Large
tokenizer = AutoTokenizer.from_pretrained("grammarly/coedit-large")
model = AutoModelForSeq2SeqLM.from_pretrained("grammarly/coedit-large").to(device)
for text in text1:
    print(f"Original: {text}")
    paraphrased = paraphrase(text)
    for p in paraphrased:
        print(f"Paraphrased: {p}")
    print("\n")


Original: Today is our dragon boat festival, in our Chinese culture, to celebrate it with all safe and great in our lives.
Paraphrased: Today, in our Chinese culture, we celebrate the Dragon Boat Festival to welcome all who are safe and good.
Paraphrased: Our Dragon Boat Festival, our Chinese culture, celebrating it with all the safest and best people in our lives.
Paraphrased: This is our Dragon Boat Festival in Chinese culture, celebrating it with all the safe and wonderful people in our lives.


Original: Hope you too, to enjoy it as my deepest wishes.
Paraphrased: I hope you'll enjoy it as much as I wish.
Paraphrased: It is my sincere wish that you will enjoy it as much as I do.
Paraphrased: Hopefully, I'll be able to enjoy it as much as I wish.


Original: Thank your message to show our words to the doctor, as his next contract checking, to all of us.
Paraphrased: Thank you for your message that we could show the doctor our words as part of his next contract check.
Paraphrased: Yo

Raheja, V., Kumar, D., Koo, R., & Kang, D. (2023). CoEdIT: Text Editing by Task-Specific Instruction Tuning. ArXiv. /abs/2305.09857

In [ ]:
# Google FLAN T5 Base
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base").to(device)
for text in text1:
    print(f"Original: {text}")
    paraphrased = paraphrase(text)
    for p in paraphrased:
        print(f"Paraphrased: {p}")
    print("\n")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


Original: Today is our dragon boat festival, in our Chinese culture, to celebrate it with all safe and great in our lives.
Paraphrased: Today is our dragon boat festival, in our Chinese culture to celebrate it with all safe and great in us lives.
Paraphrased: Today is the festival of dragon boat, in our Chinese culture to celebrate it with all safe and great in us lives.
Paraphrased: It's today in our Chinese culture to celebrate it with all safe and great in life.


Original: Hope you too, to enjoy it as my deepest wishes.
Paraphrased: I hope you too, to enjoy it as my deepest wishes.
Paraphrased: Hope you too, to enjoy it as my deepest wishes.
Paraphrased: Hopefully, you too, to enjoy it as my deepest wishes.


Original: Thank your message to show our words to the doctor, as his next contract checking, to all of us.
Paraphrased: Thank your message to show our words to the doctor, as his next contract checking, to all of us.
Paraphrased: thank you for showing our words to the doctor, 